# E-Commerce Customer Churn Prediction - Exploratory Data Analysis (EDA)
## Comprehensive Analysis and Insights

This notebook provides in-depth exploratory analysis to understand customer churn patterns, behaviors, and key drivers.

---
## Cell 1: Import Libraries
Import all necessary libraries for comprehensive EDA.

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Statistical analysis
from scipy import stats
from scipy.stats import chi2_contingency, ttest_ind

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Plot settings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

# Color palettes
churn_colors = ['#2ecc71', '#e74c3c']  # Green for not churned, Red for churned

print("✓ All libraries imported successfully!")

---
## Cell 2: Load Dataset
Load the original dataset for exploration.

In [ ]:
# Load the dataset
df = pd.read_csv('ecommerce_customer_churn_dataset.csv')

print("Dataset loaded successfully!")
print("="*80)
print(f"Dataset Shape: {df.shape}")
print(f"Total Customers: {df.shape[0]:,}")
print(f"Total Features: {df.shape[1]}")
print("="*80)

# Display first few rows
df.head(10)

---
## Cell 3: Dataset Overview
Get a comprehensive overview of the dataset structure and content.

In [ ]:
print("DATASET INFORMATION")
print("="*80)
df.info()

print("\n" + "="*80)
print("STATISTICAL SUMMARY")
print("="*80)
df.describe(include='all').T

---
## Cell 4: Target Variable Analysis
Deep dive into the churn distribution and characteristics.

In [ ]:
# Churn statistics
churn_counts = df['Churned'].value_counts()
churn_pct = df['Churned'].value_counts(normalize=True) * 100

print("CHURN ANALYSIS")
print("="*80)
print(f"Not Churned (0): {churn_counts[0]:,} customers ({churn_pct[0]:.2f}%)")
print(f"Churned (1): {churn_counts[1]:,} customers ({churn_pct[1]:.2f}%)")
print(f"\nChurn Rate: {churn_pct[1]:.2f}%")
print(f"Retention Rate: {churn_pct[0]:.2f}%")

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Count plot
sns.countplot(data=df, x='Churned', palette=churn_colors, ax=axes[0])
axes[0].set_title('Customer Churn Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Churned Status', fontsize=12)
axes[0].set_ylabel('Number of Customers', fontsize=12)
axes[0].set_xticklabels(['Not Churned', 'Churned'])
for container in axes[0].containers:
    axes[0].bar_label(container, fmt='%d')

# Pie chart
axes[1].pie(churn_counts, labels=['Not Churned', 'Churned'], autopct='%1.1f%%',
            colors=churn_colors, startangle=90, explode=(0.05, 0.05))
axes[1].set_title('Churn Percentage Distribution', fontsize=14, fontweight='bold')

# Donut chart
axes[2].pie(churn_counts, labels=['Not Churned', 'Churned'], autopct='%1.1f%%',
            colors=churn_colors, startangle=90, wedgeprops=dict(width=0.5))
axes[2].set_title('Churn Rate Overview', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n⚠️ Note: There is moderate class imbalance (71% vs 29%)")

---
## Cell 5: Missing Values Analysis
Visualize and analyze missing data patterns.

In [ ]:
# Missing values analysis
missing_data = pd.DataFrame({
    'Feature': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100
})
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values(
    'Missing_Percentage', ascending=False
).reset_index(drop=True)

print("MISSING VALUES ANALYSIS")
print("="*80)
if len(missing_data) > 0:
    print(missing_data)
    print(f"\nTotal features with missing values: {len(missing_data)}")
    print(f"Total missing values: {missing_data['Missing_Count'].sum():,}")
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Bar plot
    sns.barplot(data=missing_data, x='Feature', y='Missing_Percentage', 
                palette='Reds_r', ax=axes[0])
    axes[0].set_title('Missing Values by Feature', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Features', fontsize=12)
    axes[0].set_ylabel('Missing Percentage (%)', fontsize=12)
    axes[0].tick_params(axis='x', rotation=45)
    
    # Heatmap of missing values
    missing_matrix = df[missing_data['Feature'].tolist()].isnull().astype(int)
    sns.heatmap(missing_matrix.head(100).T, cmap='YlOrRd', cbar=True, ax=axes[1])
    axes[1].set_title('Missing Values Pattern (First 100 rows)', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Customer Index', fontsize=12)
    axes[1].set_ylabel('Features', fontsize=12)
    
    plt.tight_layout()
    plt.show()
else:
    print("✓ No missing values found!")

---
## Cell 6: Univariate Analysis - Numerical Features (Part 1)
Analyze distribution of key numerical features.

In [ ]:
# Select key numerical features for analysis
key_numerical = ['Age', 'Membership_Years', 'Login_Frequency', 'Session_Duration_Avg',
                 'Pages_Per_Session', 'Total_Purchases', 'Average_Order_Value', 'Lifetime_Value']

# Create subplots
fig, axes = plt.subplots(4, 2, figsize=(16, 18))
axes = axes.flatten()

for idx, col in enumerate(key_numerical):
    # Histogram with KDE
    df[col].dropna().hist(bins=30, alpha=0.7, color='skyblue', edgecolor='black', ax=axes[idx])
    axes[idx].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col, fontsize=10)
    axes[idx].set_ylabel('Frequency', fontsize=10)
    axes[idx].grid(axis='y', alpha=0.3)
    
    # Add statistics
    mean_val = df[col].mean()
    median_val = df[col].median()
    axes[idx].axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.2f}')
    axes[idx].axvline(median_val, color='green', linestyle='--', linewidth=2, label=f'Median: {median_val:.2f}')
    axes[idx].legend()

plt.tight_layout()
plt.show()

print("✓ Univariate analysis for key numerical features completed")

---
## Cell 7: Univariate Analysis - Numerical Features (Part 2)
Analyze engagement and behavior metrics.

In [ ]:
# Engagement and behavior features
engagement_features = ['Cart_Abandonment_Rate', 'Wishlist_Items', 'Days_Since_Last_Purchase',
                       'Email_Open_Rate', 'Customer_Service_Calls', 'Mobile_App_Usage',
                       'Returns_Rate', 'Discount_Usage_Rate']

fig, axes = plt.subplots(4, 2, figsize=(16, 18))
axes = axes.flatten()

for idx, col in enumerate(engagement_features):
    # Box plot
    df.boxplot(column=col, ax=axes[idx], patch_artist=True,
               boxprops=dict(facecolor='lightblue', color='blue'),
               medianprops=dict(color='red', linewidth=2),
               whiskerprops=dict(color='blue'),
               capprops=dict(color='blue'))
    axes[idx].set_title(f'{col} - Distribution & Outliers', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel(col, fontsize=10)
    axes[idx].grid(axis='y', alpha=0.3)
    
    # Add statistics text
    stats_text = f"Mean: {df[col].mean():.2f}\nMedian: {df[col].median():.2f}\nStd: {df[col].std():.2f}"
    axes[idx].text(0.02, 0.98, stats_text, transform=axes[idx].transAxes,
                   fontsize=9, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("✓ Engagement metrics analysis completed")

---
## Cell 8: Univariate Analysis - Categorical Features
Analyze distribution of categorical variables.

In [ ]:
# Categorical features
categorical_features = ['Gender', 'Country', 'Signup_Quarter']

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

# Gender distribution
gender_counts = df['Gender'].value_counts()
axes[0].bar(gender_counts.index, gender_counts.values, color=['#3498db', '#e74c3c'])
axes[0].set_title('Gender Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Gender', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
for i, v in enumerate(gender_counts.values):
    axes[0].text(i, v + 500, str(v), ha='center', fontweight='bold')

# Country distribution
country_counts = df['Country'].value_counts().head(10)
sns.barplot(x=country_counts.values, y=country_counts.index, palette='viridis', ax=axes[1])
axes[1].set_title('Top 10 Countries by Customer Count', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Number of Customers', fontsize=12)
axes[1].set_ylabel('Country', fontsize=12)

# Signup Quarter distribution
quarter_counts = df['Signup_Quarter'].value_counts().sort_index()
axes[2].bar(quarter_counts.index, quarter_counts.values, color=['#1abc9c', '#3498db', '#9b59b6', '#e74c3c'])
axes[2].set_title('Signup Quarter Distribution', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Quarter', fontsize=12)
axes[2].set_ylabel('Count', fontsize=12)
for i, v in enumerate(quarter_counts.values):
    axes[2].text(i, v + 200, str(v), ha='center', fontweight='bold')

# Payment Method Diversity
payment_counts = df['Payment_Method_Diversity'].value_counts().sort_index()
axes[3].bar(payment_counts.index, payment_counts.values, color='coral')
axes[3].set_title('Payment Method Diversity', fontsize=14, fontweight='bold')
axes[3].set_xlabel('Number of Payment Methods', fontsize=12)
axes[3].set_ylabel('Count', fontsize=12)

plt.tight_layout()
plt.show()

print("✓ Categorical features analysis completed")

---
## Cell 9: Bivariate Analysis - Demographics vs Churn
Analyze how demographic features relate to churn.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

# Age vs Churn
sns.boxplot(data=df, x='Churned', y='Age', palette=churn_colors, ax=axes[0])
axes[0].set_title('Age Distribution by Churn Status', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Churned (0=No, 1=Yes)', fontsize=12)
axes[0].set_ylabel('Age', fontsize=12)

# Membership Years vs Churn
sns.violinplot(data=df, x='Churned', y='Membership_Years', palette=churn_colors, ax=axes[1])
axes[1].set_title('Membership Years by Churn Status', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Churned (0=No, 1=Yes)', fontsize=12)
axes[1].set_ylabel('Membership Years', fontsize=12)

# Gender vs Churn
gender_churn = pd.crosstab(df['Gender'], df['Churned'], normalize='index') * 100
gender_churn.plot(kind='bar', stacked=False, color=churn_colors, ax=axes[2])
axes[2].set_title('Churn Rate by Gender', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Gender', fontsize=12)
axes[2].set_ylabel('Percentage (%)', fontsize=12)
axes[2].legend(['Not Churned', 'Churned'])
axes[2].tick_params(axis='x', rotation=0)

# Country vs Churn (Top 10 countries)
top_countries = df['Country'].value_counts().head(10).index
country_churn = df[df['Country'].isin(top_countries)].groupby('Country')['Churned'].mean() * 100
country_churn = country_churn.sort_values(ascending=False)
country_churn.plot(kind='barh', color='coral', ax=axes[3])
axes[3].set_title('Churn Rate by Country (Top 10)', fontsize=14, fontweight='bold')
axes[3].set_xlabel('Churn Rate (%)', fontsize=12)
axes[3].set_ylabel('Country', fontsize=12)
axes[3].axvline(x=df['Churned'].mean()*100, color='red', linestyle='--', 
                linewidth=2, label=f'Overall: {df["Churned"].mean()*100:.2f}%')
axes[3].legend()

plt.tight_layout()
plt.show()

print("✓ Demographic analysis vs Churn completed")

---
## Cell 10: Bivariate Analysis - Engagement Metrics vs Churn
Explore how customer engagement relates to churn.

In [ ]:
engagement_cols = ['Login_Frequency', 'Session_Duration_Avg', 'Pages_Per_Session', 'Mobile_App_Usage']

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, col in enumerate(engagement_cols):
    # Create violin plot
    sns.violinplot(data=df, x='Churned', y=col, palette=churn_colors, ax=axes[idx])
    axes[idx].set_title(f'{col} by Churn Status', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Churned (0=No, 1=Yes)', fontsize=12)
    axes[idx].set_ylabel(col, fontsize=12)
    
    # Add mean values as text
    mean_not_churned = df[df['Churned']==0][col].mean()
    mean_churned = df[df['Churned']==1][col].mean()
    
    stats_text = f"Not Churned: {mean_not_churned:.2f}\nChurned: {mean_churned:.2f}"
    axes[idx].text(0.02, 0.98, stats_text, transform=axes[idx].transAxes,
                   fontsize=10, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))

plt.tight_layout()
plt.show()

# Statistical significance test
print("STATISTICAL SIGNIFICANCE TEST (T-Test)")
print("="*80)
for col in engagement_cols:
    not_churned = df[df['Churned']==0][col].dropna()
    churned = df[df['Churned']==1][col].dropna()
    t_stat, p_value = ttest_ind(not_churned, churned)
    significance = "Significant" if p_value < 0.05 else "Not Significant"
    print(f"{col}: p-value = {p_value:.6f} ({significance})")

print("\n✓ Engagement metrics analysis completed")

---
## Cell 11: Bivariate Analysis - Purchase Behavior vs Churn
Analyze shopping patterns and their relationship with churn.

In [ ]:
purchase_cols = ['Total_Purchases', 'Average_Order_Value', 'Lifetime_Value', 'Cart_Abandonment_Rate']

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, col in enumerate(purchase_cols):
    # Box plot with swarm overlay
    sns.boxplot(data=df, x='Churned', y=col, palette=churn_colors, ax=axes[idx])
    axes[idx].set_title(f'{col} by Churn Status', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Churned (0=No, 1=Yes)', fontsize=12)
    axes[idx].set_ylabel(col, fontsize=12)
    
    # Calculate and display statistics
    mean_0 = df[df['Churned']==0][col].mean()
    mean_1 = df[df['Churned']==1][col].mean()
    median_0 = df[df['Churned']==0][col].median()
    median_1 = df[df['Churned']==1][col].median()
    
    stats_text = f"Not Churned - Mean: {mean_0:.2f}, Median: {median_0:.2f}\n"
    stats_text += f"Churned - Mean: {mean_1:.2f}, Median: {median_1:.2f}\n"
    stats_text += f"Difference: {abs(mean_0-mean_1):.2f}"
    
    axes[idx].text(0.02, 0.98, stats_text, transform=axes[idx].transAxes,
                   fontsize=9, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))

plt.tight_layout()
plt.show()

print("✓ Purchase behavior analysis completed")

---
## Cell 12: Bivariate Analysis - Customer Service & Satisfaction
Examine customer service interactions and satisfaction indicators.

In [ ]:
service_cols = ['Customer_Service_Calls', 'Returns_Rate', 'Email_Open_Rate', 'Product_Reviews_Written']

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, col in enumerate(service_cols):
    # Histogram by churn status
    df[df['Churned']==0][col].hist(bins=30, alpha=0.6, label='Not Churned', 
                                     color=churn_colors[0], ax=axes[idx])
    df[df['Churned']==1][col].hist(bins=30, alpha=0.6, label='Churned', 
                                     color=churn_colors[1], ax=axes[idx])
    axes[idx].set_title(f'{col} Distribution by Churn', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel(col, fontsize=12)
    axes[idx].set_ylabel('Frequency', fontsize=12)
    axes[idx].legend()
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Create summary comparison
print("\nCUSTOMER SERVICE METRICS COMPARISON")
print("="*80)
comparison_df = pd.DataFrame({
    'Metric': service_cols,
    'Not_Churned_Mean': [df[df['Churned']==0][col].mean() for col in service_cols],
    'Churned_Mean': [df[df['Churned']==1][col].mean() for col in service_cols],
})
comparison_df['Difference'] = comparison_df['Churned_Mean'] - comparison_df['Not_Churned_Mean']
comparison_df['Percent_Change'] = (comparison_df['Difference'] / comparison_df['Not_Churned_Mean']) * 100

print(comparison_df.to_string(index=False))
print("\n✓ Customer service analysis completed")

---
## Cell 13: Correlation Analysis
Examine correlations between numerical features.

In [ ]:
# Select numerical features for correlation
numerical_features = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Calculate correlation matrix
correlation_matrix = df[numerical_features].corr()

# Create heatmap
plt.figure(figsize=(18, 14))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Heatmap - All Numerical Features', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Find top correlations with Churn
churn_correlation = correlation_matrix['Churned'].sort_values(ascending=False)
print("\nTOP CORRELATIONS WITH CHURN")
print("="*80)
print(churn_correlation)

# Visualize top correlations
top_corr = churn_correlation[1:11]  # Exclude Churned itself
plt.figure(figsize=(12, 6))
colors_corr = ['green' if x > 0 else 'red' for x in top_corr.values]
plt.barh(top_corr.index, top_corr.values, color=colors_corr)
plt.xlabel('Correlation with Churn', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.title('Top 10 Features Correlated with Churn', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
plt.tight_layout()
plt.show()

print("\n✓ Correlation analysis completed")

---
## Cell 14: Customer Segmentation Analysis
Segment customers based on value and activity levels.

In [ ]:
# Create customer segments
df['Value_Segment'] = pd.cut(df['Lifetime_Value'], 
                              bins=[0, 1000, 2000, 3000, float('inf')],
                              labels=['Low', 'Medium', 'High', 'Premium'])

df['Activity_Segment'] = pd.cut(df['Login_Frequency'],
                                bins=[0, 5, 15, 25, float('inf')],
                                labels=['Inactive', 'Low', 'Medium', 'High'])

# Analyze churn by segments
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Churn by Value Segment
value_churn = df.groupby('Value_Segment')['Churned'].agg(['mean', 'count'])
value_churn['mean'] *= 100
axes[0, 0].bar(value_churn.index, value_churn['mean'], color='coral')
axes[0, 0].set_title('Churn Rate by Customer Value Segment', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Value Segment', fontsize=12)
axes[0, 0].set_ylabel('Churn Rate (%)', fontsize=12)
for i, v in enumerate(value_churn['mean']):
    axes[0, 0].text(i, v + 1, f'{v:.1f}%', ha='center', fontweight='bold')

# Customer count by Value Segment
axes[0, 1].bar(value_churn.index, value_churn['count'], color='skyblue')
axes[0, 1].set_title('Customer Count by Value Segment', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Value Segment', fontsize=12)
axes[0, 1].set_ylabel('Number of Customers', fontsize=12)

# Churn by Activity Segment
activity_churn = df.groupby('Activity_Segment')['Churned'].agg(['mean', 'count'])
activity_churn['mean'] *= 100
axes[1, 0].bar(activity_churn.index, activity_churn['mean'], color='lightgreen')
axes[1, 0].set_title('Churn Rate by Activity Segment', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Activity Segment', fontsize=12)
axes[1, 0].set_ylabel('Churn Rate (%)', fontsize=12)
for i, v in enumerate(activity_churn['mean']):
    axes[1, 0].text(i, v + 1, f'{v:.1f}%', ha='center', fontweight='bold')

# 2D Segmentation: Value vs Activity
segment_churn = df.groupby(['Value_Segment', 'Activity_Segment'])['Churned'].mean().unstack() * 100
sns.heatmap(segment_churn, annot=True, fmt='.1f', cmap='RdYlGn_r', 
            cbar_kws={'label': 'Churn Rate (%)'}, ax=axes[1, 1])
axes[1, 1].set_title('Churn Rate: Value vs Activity Segments', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Activity Segment', fontsize=12)
axes[1, 1].set_ylabel('Value Segment', fontsize=12)

plt.tight_layout()
plt.show()

print("✓ Customer segmentation analysis completed")

---
## Cell 15: RFM Analysis (Recency, Frequency, Monetary)
Perform RFM analysis to understand customer value.

In [ ]:
# Create RFM scores
df['Recency_Score'] = pd.qcut(df['Days_Since_Last_Purchase'], q=4, labels=[4, 3, 2, 1], duplicates='drop')
df['Frequency_Score'] = pd.qcut(df['Total_Purchases'].rank(method='first'), q=4, labels=[1, 2, 3, 4], duplicates='drop')
df['Monetary_Score'] = pd.qcut(df['Lifetime_Value'].rank(method='first'), q=4, labels=[1, 2, 3, 4], duplicates='drop')

# Calculate overall RFM score
df['RFM_Score'] = (df['Recency_Score'].astype(int) + 
                   df['Frequency_Score'].astype(int) + 
                   df['Monetary_Score'].astype(int))

# Categorize customers based on RFM score
df['Customer_Category'] = pd.cut(df['RFM_Score'],
                                bins=[0, 4, 7, 10, 12],
                                labels=['At Risk', 'Potential', 'Loyal', 'Champion'])

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# RFM Score Distribution
df['RFM_Score'].hist(bins=12, color='purple', alpha=0.7, edgecolor='black', ax=axes[0, 0])
axes[0, 0].set_title('RFM Score Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('RFM Score', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)

# Customer Categories
category_counts = df['Customer_Category'].value_counts()
axes[0, 1].pie(category_counts, labels=category_counts.index, autopct='%1.1f%%',
               colors=['#e74c3c', '#f39c12', '#3498db', '#2ecc71'], startangle=90)
axes[0, 1].set_title('Customer Categories Distribution', fontsize=14, fontweight='bold')

# Churn Rate by Customer Category
category_churn = df.groupby('Customer_Category')['Churned'].mean() * 100
category_churn = category_churn.sort_values(ascending=False)
axes[1, 0].barh(category_churn.index, category_churn.values, 
                color=['#e74c3c', '#f39c12', '#3498db', '#2ecc71'])
axes[1, 0].set_title('Churn Rate by Customer Category', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Churn Rate (%)', fontsize=12)
axes[1, 0].set_ylabel('Customer Category', fontsize=12)
for i, v in enumerate(category_churn.values):
    axes[1, 0].text(v + 1, i, f'{v:.1f}%', va='center', fontweight='bold')

# RFM Components Comparison
rfm_components = pd.DataFrame({
    'Not Churned': [df[df['Churned']==0]['Recency_Score'].astype(int).mean(),
                    df[df['Churned']==0]['Frequency_Score'].astype(int).mean(),
                    df[df['Churned']==0]['Monetary_Score'].astype(int).mean()],
    'Churned': [df[df['Churned']==1]['Recency_Score'].astype(int).mean(),
                df[df['Churned']==1]['Frequency_Score'].astype(int).mean(),
                df[df['Churned']==1]['Monetary_Score'].astype(int).mean()]
}, index=['Recency', 'Frequency', 'Monetary'])

rfm_components.plot(kind='bar', ax=axes[1, 1], color=churn_colors)
axes[1, 1].set_title('RFM Components by Churn Status', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('RFM Component', fontsize=12)
axes[1, 1].set_ylabel('Average Score', fontsize=12)
axes[1, 1].legend(['Not Churned', 'Churned'])
axes[1, 1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

print("\nRFM ANALYSIS SUMMARY")
print("="*80)
print(df.groupby('Customer_Category').agg({
    'Churned': ['count', 'mean'],
    'Lifetime_Value': 'mean',
    'Total_Purchases': 'mean'
}).round(2))

print("\n✓ RFM analysis completed")

---
## Cell 16: Time-Based Analysis
Analyze churn patterns over time dimensions.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Churn by Signup Quarter
quarter_churn = df.groupby('Signup_Quarter')['Churned'].agg(['mean', 'count'])
quarter_churn['mean'] *= 100
quarter_churn = quarter_churn.sort_index()

axes[0, 0].bar(quarter_churn.index, quarter_churn['mean'], color='teal')
axes[0, 0].set_title('Churn Rate by Signup Quarter', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Signup Quarter', fontsize=12)
axes[0, 0].set_ylabel('Churn Rate (%)', fontsize=12)
for i, (idx, row) in enumerate(quarter_churn.iterrows()):
    axes[0, 0].text(i, row['mean'] + 0.5, f"{row['mean']:.1f}%", ha='center', fontweight='bold')

# Days Since Last Purchase Distribution
df[df['Churned']==0]['Days_Since_Last_Purchase'].hist(bins=30, alpha=0.6, 
                                                        label='Not Churned', color=churn_colors[0], ax=axes[0, 1])
df[df['Churned']==1]['Days_Since_Last_Purchase'].hist(bins=30, alpha=0.6, 
                                                        label='Churned', color=churn_colors[1], ax=axes[0, 1])
axes[0, 1].set_title('Days Since Last Purchase by Churn', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Days Since Last Purchase', fontsize=12)
axes[0, 1].set_ylabel('Frequency', fontsize=12)
axes[0, 1].legend()

# Membership Years vs Churn
membership_churn = df.groupby(pd.cut(df['Membership_Years'], bins=5))['Churned'].mean() * 100
membership_churn.plot(kind='bar', color='orange', ax=axes[1, 0])
axes[1, 0].set_title('Churn Rate by Membership Duration', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Membership Years Range', fontsize=12)
axes[1, 0].set_ylabel('Churn Rate (%)', fontsize=12)
axes[1, 0].tick_params(axis='x', rotation=45)

# Recency vs Total Purchases (colored by Churn)
sample_df = df.sample(n=min(5000, len(df)), random_state=42)
scatter = axes[1, 1].scatter(sample_df['Days_Since_Last_Purchase'], 
                             sample_df['Total_Purchases'],
                             c=sample_df['Churned'], 
                             cmap='RdYlGn_r', alpha=0.5, s=20)
axes[1, 1].set_title('Recency vs Purchase Frequency (Colored by Churn)', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Days Since Last Purchase', fontsize=12)
axes[1, 1].set_ylabel('Total Purchases', fontsize=12)
plt.colorbar(scatter, ax=axes[1, 1], label='Churned')

plt.tight_layout()
plt.show()

print("✓ Time-based analysis completed")

---
## Cell 17: Feature Importance Insights
Identify which features are most predictive of churn.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Prepare data for quick feature importance analysis
df_temp = df.copy()

# Encode categorical variables
le = LabelEncoder()
for col in ['Gender', 'Country', 'Signup_Quarter']:
    df_temp[col + '_encoded'] = le.fit_transform(df_temp[col].astype(str))

# Select features (numerical + encoded)
feature_cols = df_temp.select_dtypes(include=['float64', 'int64']).columns.tolist()
feature_cols = [col for col in feature_cols if col not in ['Churned']]

# Fill missing values
X_temp = df_temp[feature_cols].fillna(df_temp[feature_cols].median())
y_temp = df_temp['Churned']

# Train a quick Random Forest for feature importance
rf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10, n_jobs=-1)
rf.fit(X_temp, y_temp)

# Get feature importances
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

# Visualize top 20 features
top_features = feature_importance.head(20)

plt.figure(figsize=(12, 8))
sns.barplot(data=top_features, x='Importance', y='Feature', palette='viridis')
plt.title('Top 20 Most Important Features for Churn Prediction', fontsize=16, fontweight='bold')
plt.xlabel('Feature Importance Score', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.tight_layout()
plt.show()

print("\nTOP 20 FEATURES BY IMPORTANCE")
print("="*80)
print(top_features.to_string(index=False))

print("\n✓ Feature importance analysis completed")

---
## Cell 18: Key Business Insights
Summarize actionable insights from the analysis.

In [ ]:
print("="*80)
print("KEY BUSINESS INSIGHTS FROM EDA")
print("="*80)

insights = """
📊 CHURN OVERVIEW:
   - Overall churn rate: {churn_rate:.2f}%
   - {churned_count:,} customers churned out of {total:,} total customers
   - Class imbalance present: Need to address in modeling

🎯 TOP CHURN INDICATORS:
   1. LOW ENGAGEMENT:
      - Churned customers have {login_diff:.1f}% lower login frequency
      - {session_diff:.1f}% shorter session durations
      - Recommendation: Implement engagement campaigns for low-activity users
   
   2. SHOPPING BEHAVIOR:
      - Churned customers: {cart_churned:.1f}% cart abandonment vs {cart_not:.1f}% (not churned)
      - {purchases_diff:.1f} fewer purchases on average
      - Recommendation: Send cart abandonment emails, offer incentives
   
   3. RECENCY MATTERS:
      - Churned customers haven't purchased in {days_churned:.1f} days on average
      - vs {days_not:.1f} days for retained customers
      - Recommendation: Re-engagement campaigns after 30+ days of inactivity
   
   4. CUSTOMER VALUE:
      - Low-value customers churn at higher rates
      - "At Risk" segment has highest churn
      - Recommendation: VIP programs for high-value customers

💡 CUSTOMER SEGMENTATION INSIGHTS:
   - Champions: Lowest churn ({champion_churn:.1f}%) - Focus on retention
   - At Risk: Highest churn ({risk_churn:.1f}%) - Immediate intervention needed
   - Inactive users (0-5 logins): {inactive_churn:.1f}% churn rate
   - High activity users (25+ logins): {active_churn:.1f}% churn rate

🌍 GEOGRAPHICAL INSIGHTS:
   - Churn rates vary by country (range: {min_country_churn:.1f}% - {max_country_churn:.1f}%)
   - Consider localized retention strategies

⏰ TIMING INSIGHTS:
   - Membership duration inversely correlates with churn
   - First-year customers need extra attention
   - Signup quarter shows variation in churn rates

🔧 ACTIONABLE RECOMMENDATIONS:
   1. Implement early warning system for customers with:
      - Low login frequency (<5 per month)
      - High cart abandonment (>60%)
      - No purchase in 30+ days
   
   2. Create targeted interventions:
      - Personalized email campaigns
      - Special discounts for at-risk customers
      - Mobile app push notifications for inactive users
   
   3. Focus on customer service:
      - Reduce service calls through better UX
      - Proactive support for customers with issues
   
   4. Enhance engagement:
      - Loyalty programs
      - Gamification features
      - Personalized product recommendations
""".format(
    churn_rate=df['Churned'].mean() * 100,
    churned_count=df['Churned'].sum(),
    total=len(df),
    login_diff=((df[df['Churned']==0]['Login_Frequency'].mean() - 
                 df[df['Churned']==1]['Login_Frequency'].mean()) / 
                 df[df['Churned']==0]['Login_Frequency'].mean()) * 100,
    session_diff=((df[df['Churned']==0]['Session_Duration_Avg'].mean() - 
                   df[df['Churned']==1]['Session_Duration_Avg'].mean()) / 
                   df[df['Churned']==0]['Session_Duration_Avg'].mean()) * 100,
    cart_churned=df[df['Churned']==1]['Cart_Abandonment_Rate'].mean(),
    cart_not=df[df['Churned']==0]['Cart_Abandonment_Rate'].mean(),
    purchases_diff=df[df['Churned']==0]['Total_Purchases'].mean() - 
                   df[df['Churned']==1]['Total_Purchases'].mean(),
    days_churned=df[df['Churned']==1]['Days_Since_Last_Purchase'].mean(),
    days_not=df[df['Churned']==0]['Days_Since_Last_Purchase'].mean(),
    champion_churn=df[df['Customer_Category']=='Champion']['Churned'].mean() * 100,
    risk_churn=df[df['Customer_Category']=='At Risk']['Churned'].mean() * 100,
    inactive_churn=df[df['Login_Frequency'] <= 5]['Churned'].mean() * 100,
    active_churn=df[df['Login_Frequency'] >= 25]['Churned'].mean() * 100,
    min_country_churn=df.groupby('Country')['Churned'].mean().min() * 100,
    max_country_churn=df.groupby('Country')['Churned'].mean().max() * 100
)

print(insights)

print("\n" + "="*80)
print("✓ EDA COMPLETED - Ready for Machine Learning Modeling")
print("="*80)

---
## Cell 19: Export EDA Summary Report
Save key statistics and insights for reference.

In [ ]:
# Create comprehensive summary report
summary_report = {}

# Overall statistics
summary_report['Overall'] = {
    'Total_Customers': len(df),
    'Churn_Rate_%': df['Churned'].mean() * 100,
    'Retention_Rate_%': (1 - df['Churned'].mean()) * 100
}

# Feature statistics by churn
numerical_features = ['Age', 'Membership_Years', 'Login_Frequency', 'Total_Purchases', 
                     'Average_Order_Value', 'Lifetime_Value', 'Cart_Abandonment_Rate']

for feature in numerical_features:
    summary_report[feature] = {
        'Not_Churned_Mean': df[df['Churned']==0][feature].mean(),
        'Churned_Mean': df[df['Churned']==1][feature].mean(),
        'Difference': df[df['Churned']==0][feature].mean() - df[df['Churned']==1][feature].mean()
    }

# Convert to DataFrame and save
summary_df = pd.DataFrame(summary_report).T
summary_df.to_csv('EDA_Summary_Report.csv')

print("EDA SUMMARY REPORT")
print("="*80)
print(summary_df)

print("\n✓ Summary report saved as 'EDA_Summary_Report.csv'")
print("\n📊 EXPLORATORY DATA ANALYSIS COMPLETE!")

---
## Summary

This comprehensive EDA notebook analyzed:
- ✓ Dataset structure and quality
- ✓ Target variable distribution and class balance
- ✓ Univariate analysis of all features
- ✓ Bivariate relationships with churn
- ✓ Correlation patterns
- ✓ Customer segmentation (RFM analysis)
- ✓ Time-based patterns
- ✓ Feature importance rankings
- ✓ Actionable business insights

**Next Steps:** Use these insights to build predictive models in the next notebook!